In [1]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *

Create and play with the properties of a quantum object

In [2]:
# Create a random operator as a 2x2 square matrix
Operator1 = Qobj([[1, 2], [2, 4]])
print(Operator1)

# Create a random state as a 2 vector
State1 = Qobj([[1], [2]])
print(State1)

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[1. 2.]
 [2. 4.]]
Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
Qobj data =
[[1.]
 [2.]]


In [3]:
# Generate some useful operators

H = Qobj([[1, 2, 3, 4], [1, 2, 3, 4],[1, 2, 3, 4],[1, 2, 3, 4]])
e, s = H.groundstate()

c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\qutip\core\qobj.py:1626: UserWarning: Ground state may be degenerate.
  warnings.warn("Ground state may be degenerate.", UserWarning)


In [4]:
print(e, s)

0j Quantum object: dims=[[4], [1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[ 0.98198051]
 [-0.10910895]
 [-0.10910895]
 [-0.10910895]]


In [5]:
#   Tensor notation
g = basis(2, 0)     # Ground state is (1, 0), basis(2,0) means a "2" dim vector with 1 being at the "0"th index
n = basis(2, 1)

a = destroy(2)      # Make the annihilation and excitation operators
sig_p = create(2)

e = (sig_p*g)       # Act excitation on ground
n_1 = (a*n)         # Act annihilation on n photons


print('atom g', g.full())
print('light n', n.full())
print('atom e', e.full())
print('light n-1', n_1.full())

e_n_1 = tensor(e, n_1)  # Encode by making this |e, n-1>


# We can achieve the same e_n_1 as above directly using tensors:
# instead of applying sig_p on g and a on n directly, I make sig_p X a and g X n
# (where X is tensor product) and apply one to the other
g_n = tensor(g, n)
e_n_1_tens = tensor(sig_p, a)*g_n


# Comparing the results to find they are the same
print('tens', e_n_1_tens.full())
print('no tens', e_n_1.full())

atom g [[1.+0.j]
 [0.+0.j]]
light n [[0.+0.j]
 [1.+0.j]]
atom e [[0.+0.j]
 [1.+0.j]]
light n-1 [[1.+0.j]
 [0.+0.j]]
tens [[0.+0.j]
 [0.+0.j]
 [1.+0.j]
 [0.+0.j]]
no tens [[0.+0.j]
 [0.+0.j]
 [1.+0.j]
 [0.+0.j]]


In [6]:
# Making the hamiltonian

# N = number of photons, thus number of "possible" atom states, but of course only
# two is accessible (ground and n), and we do not allow further excitation. Thus
# still work in 2 dimensions, but N is used for scaling?
N = 10
a = destroy(2)
a_dag = a.dag()
sig_p = create(2)
sig_m = destroy(2)
sigma_z = tensor(sigmaz(), qeye(2)) # making sigmaz a tensor that only applies on the atom, not light
                                  # because qeye is the identity matrix, and in this program:
                                  # (A X B)(gXn) --> Ag X Bn if A and B applies on g and n only, respectively
Count_op = tensor(qeye(2), a_dag*a)

wa = 1 # atom frequency
wc = 2 # photon frequency
Omega = 1 # Rabi oscillation frequency


H_JC = wa/2 * sigma_z + wc*Count_op + Omega/2 * (tensor(sig_p, a) + tensor(sig_m, a_dag))

print(H_JC.eigenstates())
print(H_JC.eigenenergies())


(array([-0.58113883,  0.5       ,  1.5       ,  2.58113883]), array([Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
       Qobj data =
       [[ 0.        ]
        [-0.16018224]
        [ 0.98708746]
        [ 0.        ]]                                                             ,
       Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
       Qobj data =
       [[1.]
        [0.]
        [0.]
        [0.]]                                                                      ,
       Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
       Qobj data =
       [[0.]
        [0.]
        [0.]
        [1.]]                                                                      ,
       Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
       Qobj data =
       [[0.        ]
        [0.98708746]
        [0.16018224]
        [0.        ]]                                           

In [7]:
print(Count_op * g_n)

Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.]
 [1.]
 [0.]
 [0.]]


In [8]:
sig = tensor(sigmaz(), qeye(2))
print(e_n_1)

print(sig*e_n_1)

Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.]
 [0.]
 [1.]
 [0.]]
Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[ 0.]
 [ 0.]
 [-1.]
 [ 0.]]


In [9]:
print(g_n)

Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.]
 [1.]
 [0.]
 [0.]]


In [10]:
print(e_n_1)

Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.]
 [0.]
 [1.]
 [0.]]


Making N photon states (ofc only 2 is accessible)

In [63]:
N = 2 # number of photon states (number of photons + 1(no photon state) )
g = basis(2, 0)
n = basis(N, N-1)

e = basis(2, 1)
n_1 = basis(N, N-2)

g_n = tensor(g, n)
e_n_1 = tensor(e, n_1)


sigma_z = tensor(sigmaz(), qeye(N))
a = tensor(qeye(2), destroy(N))
a_dag = a.dag()
sig_p = tensor(create(2), qeye(N))
sig_m = sig_p.dag()

int_excitation = tensor(create(2), destroy(N))
int_deexcitation = tensor(destroy(2), create(N))
int_perturn = tensor(destroy(2), destroy(N)) + tensor(create(2), create(N))



print('-------------- |e,n-1> ----------------')
print('Original state : ')
print(e_n_1)
print('acted on state :')
print((a_dag*a)*e_n_1)


print('-------------- |g,n> ------------------')
print(sig_m*g_n)
print((a_dag*a)*g_n)
print(n)

-------------- |e,n-1> ----------------
Original state : 
Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.]
 [0.]
 [1.]
 [0.]]
acted on state :
Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.]
 [0.]
 [0.]
 [0.]]
-------------- |g,n> ------------------
Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.]
 [0.]
 [0.]
 [0.]]
Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.]
 [1.]
 [0.]
 [0.]]
Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
Qobj data =
[[0.]
 [1.]]


In [46]:
wa = 2.5
wc = 0.5
Omega = 3
H_JC = wa/2 * sigma_z + wc * a_dag*a + Omega/2 * (int_excitation + int_deexcitation + int_perturn)

# print(H_JC.eigenstates())
# print(g_n)
# print(e_n_1)

In [49]:
print(H_JC.eigenenergies()[29])  #e, 10

9.74389685847022


In [ ]:
print(H_JC.eigenenergies()[19])  # e, 10

11.031605496035798


In [ ]:
from prettytable import PrettyTable

matrix = np.round(np.real(H_JC.full()))
table = PrettyTable()

for row in matrix:
    table.add_row(row)


+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
| Field 1 | Field 2 | Field 3 | Field 4 | Field 5 | Field 6 | Field 7 | Field 8 | Field 9 | Field 10 | Field 11 | Field 12 | Field 13 | Field 14 | Field 15 | Field 16 | Field 17 | Field 18 | Field 19 | Field 20 |
+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|   1.0   |   0.0   |   0.0   |   0.0   |   0.0   |   0.0   |   0.0   |   0.0   |   0.0   |   0.0    |   0.0    |   2.0    |   0.0    |   0.0    |   0.0    |   0.0    |   0.0    |   0.0    |   0.0    |   0.0    |
|   0.0   |   2.0   |   0.0   |   0.0   |   0.0   |   0.0   |   0.0   |   0.0   |   0.0   |   0.0    |   2.0    |   0.0    |   2.0    |   0.0    |  

In [64]:
from prettytable import PrettyTable
def Hjc (N, Omega):
    # N = number of photon states (number of photons + 1(no photon state) )
    sigma_z = tensor(sigmaz(), qeye(N))
    a = tensor(qeye(2), destroy(N))
    a_dag = a.dag()
    sig_p = tensor(create(2), qeye(N))
    sig_m = sig_p.dag()

    int_excitation = tensor(create(2), destroy(N))
    int_deexcitation = tensor(destroy(2), create(N))
    int_perturn = tensor(destroy(2), destroy(N)) + tensor(create(2), create(N))

    return wa/2 * sigma_z + wc * a_dag*a + Omega/2 * (int_excitation + int_deexcitation + int_perturn)


matrix = np.real(Hjc(3).full())
table = PrettyTable()

for row in matrix:
    table.add_row(row)
print(table)

TypeError: Hjc() missing 1 required positional argument: 'Omega'

In [ ]:
Oms = np.linspace(0, 5, 10)
Hams = [Hjc(2, i)

TypeError: unsupported operand type(s) for *: 'numpy.ndarray' and 'Qobj'